### Model Stacking

#### Data and Model Loading

In [1]:
# import libraries used

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import  OneHotEncoder, LabelEncoder

import swifter
import gc

In [2]:
# Load processed dataset.

data = pd.read_pickle('v30_data.pkl')
data_embed = pd.read_pickle('v30_data_embed.pkl')

In [3]:
import pickle

xgb_on_sale = pickle.load(open("v30_xgb_on_sale.pkl", "rb"))
xgb_on_sale_hl = pickle.load(open("v30_xgb_on_sale_hl.pkl", "rb"))
xgb_first_sale = pickle.load(open("v30_xgb_first_sale.pkl", "rb"))
xgb_first_sale_hl = pickle.load(open("v30_xgb_first_sale_hl.pkl", "rb"))

In [4]:
lgb_on_sale = pickle.load(open("v30_lgb_on_sale.pkl", "rb"))
lgb_on_sale_hl = pickle.load(open("v30_lgb_on_sale_hl.pkl", "rb"))
lgb_first_sale = pickle.load(open("v30_lgb_first_sale.pkl", "rb"))
lgb_first_sale_hl = pickle.load(open("v30_lgb_first_sale_hl.pkl", "rb"))

In [5]:
# train, validation, test data set parameter setting.

test_date_block = 34
val_date_block = test_date_block - 1
train_date_block = val_date_block - 1

In [6]:
high_loss_shops = [20, 9, 25, 31, 42, 28, 55]
#normal_shops = list(set(data.shop_id.unique()) - set(high_loss_shops))
normal_shops = list(set(data.shop_id.unique()))

#### lgb Data Loading

In [7]:
on_sale_data = data.copy()

In [8]:
drop_list_on_sale = [
    'days',
    'shop_city',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    'shop_item_on_market',
    #'shop_item_no_sale',
    #'avg_date_item_price_cnt_lag_1',
    'avg_date_item_price_cnt_lag_2',
    'avg_date_item_price_cnt_lag_3',
    #'avg_date_shop_item_price_cnt_lag_1',
    'avg_date_shop_item_price_cnt_lag_2',
    'avg_date_shop_item_price_cnt_lag_3',
    'avg_date_cat_price_cnt_lag_1',
    'avg_date_cat_price_cnt_lag_2',
    'avg_date_cat_price_cnt_lag_3',
    'avg_date_shop_cat_price_cnt_lag_1',
    'avg_date_shop_cat_price_cnt_lag_2',
    'avg_date_shop_cat_price_cnt_lag_3',
    'date_item_price_lag_2',
    'date_item_price_lag_3',
    'date_shop_item_price_lag_2',
    'date_shop_item_price_lag_3',
    #'date_shop_revenue_lag_1',
    'date_shop_revenue_lag_2',
    'date_shop_revenue_lag_3',
    #'date_shop_cat_revenue_lag_1',
    'date_shop_cat_revenue_lag_2',
    'date_shop_cat_revenue_lag_3',
]

on_sale_data.drop(drop_list_on_sale, axis=1, inplace=True)

In [9]:
on_sale_data = on_sale_data[on_sale_data.time_on_market > 0]

X_valid_on_sale = on_sale_data[(on_sale_data.date_block_num == val_date_block) & \
                           on_sale_data.shop_id.isin(normal_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_on_sale = on_sale_data[(on_sale_data.date_block_num == val_date_block) & \
                           on_sale_data.shop_id.isin(normal_shops)]['date_shop_item_cnt']

X_valid_on_sale_hl = on_sale_data[(on_sale_data.date_block_num == val_date_block)& \
                           on_sale_data.shop_id.isin(high_loss_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_on_sale_hl = on_sale_data[(on_sale_data.date_block_num == val_date_block)& \
                           on_sale_data.shop_id.isin(high_loss_shops)]['date_shop_item_cnt']

In [10]:
first_sale_data = data.copy()

In [11]:
drop_list_first_sale = [
    'days',
    'shop_city',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    #'time_on_market',
    #'time_no_sale',
    'shop_item_on_market',
    #'shop_item_no_sale',
    'date_shop_item_cnt_lag_1',
    'date_shop_item_cnt_lag_2',
    'date_shop_item_cnt_lag_3',
    'avg_date_item_cnt_lag_1',
    'avg_date_item_cnt_lag_2',
    'avg_date_item_cnt_lag_3',
    'avg_date_shop_type_item_cnt_lag_1',
    'avg_date_shop_type_item_cnt_lag_2',
    'avg_date_shop_type_item_cnt_lag_3',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    'avg_date_item_price_cnt_lag_1',
    'avg_date_item_price_cnt_lag_2',
    'avg_date_item_price_cnt_lag_3',
    'avg_date_shop_item_price_cnt_lag_1',
    'avg_date_shop_item_price_cnt_lag_2',
    'avg_date_shop_item_price_cnt_lag_3',
    'avg_date_cat_price_cnt_lag_1',
    'avg_date_cat_price_cnt_lag_2',
    'avg_date_cat_price_cnt_lag_3',
    'avg_date_shop_cat_price_cnt_lag_1',
    'avg_date_shop_cat_price_cnt_lag_2',
    'avg_date_shop_cat_price_cnt_lag_3',
    'date_item_price_lag_1',
    'date_item_price_lag_2',
    'date_item_price_lag_3',
    'date_shop_item_price_lag_1',
    'date_shop_item_price_lag_2',
    'date_shop_item_price_lag_3',
    #'date_shop_revenue_lag_1',
    'date_shop_revenue_lag_2',
    'date_shop_revenue_lag_3',
    #'date_shop_cat_revenue_lag_1',
    'date_shop_cat_revenue_lag_2',
    'date_shop_cat_revenue_lag_3',
]

first_sale_data.drop(drop_list_first_sale, axis=1, inplace=True)

In [12]:
first_sale_data = first_sale_data[(first_sale_data.time_on_market < 3)]

In [13]:
X_valid_first_sale = first_sale_data[(first_sale_data.date_block_num == val_date_block) & \
                           (first_sale_data.time_on_market == 0) & \
                           first_sale_data.shop_id.isin(normal_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_first_sale = first_sale_data[(first_sale_data.date_block_num == val_date_block) & \
                           (first_sale_data.time_on_market == 0) & \
                           first_sale_data.shop_id.isin(normal_shops)]['date_shop_item_cnt']

In [14]:
X_valid_first_sale_ft = X_valid_first_sale.drop(['item_id'], axis=1)
Y_valid_first_sale_ft = Y_valid_first_sale.copy()

In [15]:
X_valid_first_sale_hl = first_sale_data[(first_sale_data.date_block_num == val_date_block) & \
                               (first_sale_data.time_on_market == 0) & \
                               first_sale_data.shop_id.isin(high_loss_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_first_sale_hl = first_sale_data[(first_sale_data.date_block_num == val_date_block) & \
                               (first_sale_data.time_on_market == 0)  & \
                               first_sale_data.shop_id.isin(high_loss_shops)]['date_shop_item_cnt']

In [16]:
X_valid_first_sale_hl_ft = X_valid_first_sale_hl.drop(['item_id'], axis=1)
Y_valid_first_sale_hl_ft = Y_valid_first_sale_hl.copy()

#### xgb Data Loading

In [19]:
on_sale_data_em = data_embed.copy()

In [20]:
drop_list_on_sale = [
    'days',
    'shop_city',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    'shop_item_on_market',
    #'shop_item_no_sale',
    #'avg_date_item_price_cnt_lag_1',
    'avg_date_item_price_cnt_lag_2',
    'avg_date_item_price_cnt_lag_3',
    #'avg_date_shop_item_price_cnt_lag_1',
    'avg_date_shop_item_price_cnt_lag_2',
    'avg_date_shop_item_price_cnt_lag_3',
    'avg_date_cat_price_cnt_lag_1',
    'avg_date_cat_price_cnt_lag_2',
    'avg_date_cat_price_cnt_lag_3',
    'avg_date_shop_cat_price_cnt_lag_1',
    'avg_date_shop_cat_price_cnt_lag_2',
    'avg_date_shop_cat_price_cnt_lag_3',
    'date_item_price_lag_2',
    'date_item_price_lag_3',
    'date_shop_item_price_lag_2',
    'date_shop_item_price_lag_3',
    #'date_shop_revenue_lag_1',
    'date_shop_revenue_lag_2',
    'date_shop_revenue_lag_3',
    #'date_shop_cat_revenue_lag_1',
    'date_shop_cat_revenue_lag_2',
    'date_shop_cat_revenue_lag_3',
]

on_sale_data_em.drop(drop_list_on_sale, axis=1, inplace=True)

In [21]:
on_sale_data_em = on_sale_data_em[on_sale_data_em.time_on_market > 0]

In [22]:
X_valid_on_sale_em = on_sale_data_em[(on_sale_data_em.date_block_num == val_date_block) & \
                           on_sale_data_em.shop_id.isin(normal_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_on_sale_em = on_sale_data_em[(on_sale_data_em.date_block_num == val_date_block) & \
                           on_sale_data_em.shop_id.isin(normal_shops)]['date_shop_item_cnt']
X_valid_on_sale_hl_em = on_sale_data_em[(on_sale_data_em.date_block_num == val_date_block)& \
                           on_sale_data_em.shop_id.isin(high_loss_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_on_sale_hl_em = on_sale_data_em[(on_sale_data_em.date_block_num == val_date_block)& \
                           on_sale_data_em.shop_id.isin(high_loss_shops)]['date_shop_item_cnt']

In [23]:
embed_drop_list = [
    'date_block_num',
    'shop_id',
    'shop_type',
    'item_category_id', 
    'cat_type',
    'sub_type',
    'month',
]

X_valid_on_sale_em.drop(embed_drop_list, axis=1, inplace=True)
X_valid_on_sale_hl_em.drop(embed_drop_list, axis=1, inplace=True)

In [40]:
first_sale_data_em = data_embed.copy()

In [41]:
drop_list_first_sale = [
    'days',
    'shop_city',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    #'time_on_market',
    #'time_no_sale',
    'shop_item_on_market',
    #'shop_item_no_sale',
    'date_shop_item_cnt_lag_1',
    'date_shop_item_cnt_lag_2',
    'date_shop_item_cnt_lag_3',
    'avg_date_item_cnt_lag_1',
    'avg_date_item_cnt_lag_2',
    'avg_date_item_cnt_lag_3',
    'avg_date_shop_type_item_cnt_lag_1',
    'avg_date_shop_type_item_cnt_lag_2',
    'avg_date_shop_type_item_cnt_lag_3',
    'avg_date_city_item_cnt_lag_1',
    'avg_date_city_item_cnt_lag_2',
    'avg_date_city_item_cnt_lag_3',
    'avg_date_item_price_cnt_lag_1',
    'avg_date_item_price_cnt_lag_2',
    'avg_date_item_price_cnt_lag_3',
    'avg_date_shop_item_price_cnt_lag_1',
    'avg_date_shop_item_price_cnt_lag_2',
    'avg_date_shop_item_price_cnt_lag_3',
    'avg_date_cat_price_cnt_lag_1',
    'avg_date_cat_price_cnt_lag_2',
    'avg_date_cat_price_cnt_lag_3',
    'avg_date_shop_cat_price_cnt_lag_1',
    'avg_date_shop_cat_price_cnt_lag_2',
    'avg_date_shop_cat_price_cnt_lag_3',
    'date_item_price_lag_1',
    'date_item_price_lag_2',
    'date_item_price_lag_3',
    'date_shop_item_price_lag_1',
    'date_shop_item_price_lag_2',
    'date_shop_item_price_lag_3',
    #'date_shop_revenue_lag_1',
    'date_shop_revenue_lag_2',
    'date_shop_revenue_lag_3',
    #'date_shop_cat_revenue_lag_1',
    'date_shop_cat_revenue_lag_2',
    'date_shop_cat_revenue_lag_3',
]

first_sale_data_em.drop(drop_list_first_sale, axis=1, inplace=True)

In [42]:
first_sale_data_em = first_sale_data_em[(first_sale_data_em.time_on_market < 3)]

In [43]:
X_valid_first_sale_em = first_sale_data_em[(first_sale_data_em.date_block_num == val_date_block) & \
                           (first_sale_data_em.time_on_market == 0) & \
                           first_sale_data_em.shop_id.isin(normal_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_first_sale_em = first_sale_data_em[(first_sale_data_em.date_block_num == val_date_block) & \
                           (first_sale_data_em.time_on_market == 0) & \
                           first_sale_data_em.shop_id.isin(normal_shops)]['date_shop_item_cnt']
X_valid_first_sale_hl_em = first_sale_data_em[(first_sale_data_em.date_block_num == val_date_block) & \
                               (first_sale_data_em.time_on_market == 0) & \
                               first_sale_data_em.shop_id.isin(high_loss_shops)].drop(['date_shop_item_cnt'], axis=1)
Y_valid_first_sale_hl_em = first_sale_data_em[(first_sale_data_em.date_block_num == val_date_block) & \
                               (first_sale_data_em.time_on_market == 0)  & \
                               first_sale_data_em.shop_id.isin(high_loss_shops)]['date_shop_item_cnt']

In [44]:
X_valid_first_sale_ft_em = X_valid_first_sale_em.drop(['item_id'], axis=1)
Y_valid_first_sale_ft_em = Y_valid_first_sale_em.copy()
X_valid_first_sale_hl_ft_em = X_valid_first_sale_hl_em.drop(['item_id'], axis=1)
Y_valid_first_sale_hl_ft_em = Y_valid_first_sale_hl_em.copy()

In [45]:
embed_drop_list = [
    'date_block_num',
    'shop_id',
    'shop_type',
    'item_category_id', 
    'cat_type',
    'sub_type',
    'month',
]

X_valid_first_sale_ft_em.drop(embed_drop_list, axis=1, inplace=True)
X_valid_first_sale_hl_ft_em.drop(embed_drop_list, axis=1, inplace=True)

In [46]:
len(X_valid_first_sale_ft_em.columns)

95

#### Validation

In [47]:
from sklearn.linear_model import Ridge

meta_on_sale = np.column_stack([
    lgb_on_sale.predict(X_valid_on_sale).clip(0,20),
    xgb_on_sale.predict(X_valid_on_sale_em).clip(0,20),
])

ridge_on_sale = Ridge().fit(meta_on_sale, Y_valid_on_sale)

In [48]:
meta_on_sale_hl = np.column_stack([
    lgb_on_sale_hl.predict(X_valid_on_sale_hl).clip(0,20),
    xgb_on_sale_hl.predict(X_valid_on_sale_hl_em).clip(0,20),
])

ridge_on_sale_hl = Ridge().fit(meta_on_sale_hl, Y_valid_on_sale_hl)

In [49]:
xgb_first_sale.predict(X_valid_first_sale_ft_em).clip(0,20)

array([0.01728912, 0.01728912, 0.00776045, ..., 0.44109154, 0.44109154,
       0.44109154], dtype=float32)

In [50]:
meta_first_sale = np.column_stack([
    lgb_first_sale.predict(X_valid_first_sale_ft).clip(0,20),
    xgb_first_sale.predict(X_valid_first_sale_ft_em).clip(0,20),
])

ridge_first_sale = Ridge().fit(meta_first_sale, Y_valid_first_sale)

In [51]:
meta_first_sale_hl = np.column_stack([
    lgb_first_sale_hl.predict(X_valid_first_sale_hl_ft).clip(0,20),
    xgb_first_sale_hl.predict(X_valid_first_sale_hl_ft_em).clip(0,20),
])

ridge_first_sale_hl = Ridge().fit(meta_first_sale_hl, Y_valid_first_sale_hl)

In [54]:
X_test_on_sale = on_sale_data[(on_sale_data.date_block_num == 34) & \
                              (on_sale_data.time_on_market > 0) &\
                               on_sale_data.shop_id.isin(normal_shops)]\
                             .drop(['date_shop_item_cnt'], axis=1)
X_test_on_sale_hl = on_sale_data[(on_sale_data.date_block_num == 34) & \
                              (on_sale_data.time_on_market > 0) &\
                               on_sale_data.shop_id.isin(high_loss_shops)]\
                             .drop(['date_shop_item_cnt'], axis=1)

X_test_on_sale_em = on_sale_data_em[(on_sale_data_em.date_block_num == 34) & \
                              (on_sale_data_em.time_on_market > 0) &\
                               on_sale_data_em.shop_id.isin(normal_shops)]\
                             .drop(['date_shop_item_cnt'], axis=1)
X_test_on_sale_hl_em = on_sale_data_em[(on_sale_data_em.date_block_num == 34) & \
                              (on_sale_data_em.time_on_market > 0) &\
                               on_sale_data_em.shop_id.isin(high_loss_shops)]\
                             .drop(['date_shop_item_cnt'], axis=1)
X_test_on_sale_em.drop(embed_drop_list, axis=1, inplace=True)
X_test_on_sale_hl_em.drop(embed_drop_list, axis=1, inplace=True)

meta_on_sale = np.column_stack([
    lgb_on_sale.predict(X_test_on_sale).clip(0,20),
    xgb_on_sale.predict(X_test_on_sale_em).clip(0,20),
])

meta_on_sale_hl = np.column_stack([
    lgb_on_sale_hl.predict(X_test_on_sale_hl).clip(0,20),
    xgb_on_sale_hl.predict(X_test_on_sale_hl_em).clip(0,20),
])

Y_test_on_sale = ridge_on_sale.predict(meta_on_sale).clip(0,20)
Y_test_on_sale_hl = ridge_on_sale_hl.predict(meta_on_sale_hl).clip(0,20)

X_test_on_sale = X_test_on_sale[['shop_id', 'item_id']]
X_test_on_sale['Y_test_on_sale'] = Y_test_on_sale
X_test_on_sale_hl['Y_test_on_sale_hl'] = Y_test_on_sale_hl

In [56]:
X_test_first_sale = first_sale_data[(first_sale_data.date_block_num == 34) & \
                                    (first_sale_data.time_on_market == 0) & \
                                    first_sale_data.shop_id.isin(normal_shops)] \
                                    .drop(['date_shop_item_cnt'], axis=1)
X_test_first_sale_ft = X_test_first_sale.copy()
X_test_first_sale_ft = X_test_first_sale_ft.drop(['item_id'], axis=1)

X_test_first_sale_hl = first_sale_data[(first_sale_data.date_block_num == 34) & \
                                    (first_sale_data.time_on_market == 0) & \
                                    first_sale_data.shop_id.isin(high_loss_shops)] \
                                    .drop(['date_shop_item_cnt'], axis=1)
X_test_first_sale_hl_ft = X_test_first_sale_hl.copy()
X_test_first_sale_hl_ft = X_test_first_sale_hl_ft.drop(['item_id'], axis=1)

X_test_first_sale_em = first_sale_data_em[(first_sale_data_em.date_block_num == 34) & \
                                    (first_sale_data_em.time_on_market == 0) & \
                                    first_sale_data_em.shop_id.isin(normal_shops)] \
                                    .drop(['date_shop_item_cnt'], axis=1)
X_test_first_sale_ft_em = X_test_first_sale_em.copy()
X_test_first_sale_ft_em = X_test_first_sale_ft_em.drop(['item_id'], axis=1)
X_test_first_sale_ft_em.drop(embed_drop_list, axis=1, inplace=True)

X_test_first_sale_hl_em = first_sale_data_em[(first_sale_data_em.date_block_num == 34) & \
                                    (first_sale_data_em.time_on_market == 0) & \
                                    first_sale_data_em.shop_id.isin(high_loss_shops)] \
                                    .drop(['date_shop_item_cnt'], axis=1)
X_test_first_sale_hl_ft_em = X_test_first_sale_hl_em.copy()
X_test_first_sale_hl_ft_em = X_test_first_sale_hl_ft_em.drop(['item_id'], axis=1)
X_test_first_sale_hl_ft_em.drop(embed_drop_list, axis=1, inplace=True)

meta_first_sale = np.column_stack([
    lgb_first_sale.predict(X_test_first_sale_ft).clip(0,20),
    xgb_first_sale.predict(X_test_first_sale_ft_em).clip(0,20),
])

meta_first_sale_hl = np.column_stack([
    lgb_first_sale_hl.predict(X_test_first_sale_hl_ft).clip(0,20),
    xgb_first_sale_hl.predict(X_test_first_sale_hl_ft_em).clip(0,20),
])

Y_test_first_sale = ridge_first_sale.predict(meta_first_sale).clip(0,20)
Y_test_first_sale_hl = ridge_first_sale_hl.predict(meta_first_sale_hl).clip(0,20)

X_test_first_sale = X_test_first_sale[['shop_id', 'item_id']]
X_test_first_sale['Y_test_first_sale'] = Y_test_first_sale
X_test_first_sale_hl['Y_test_first_sale_hl'] = Y_test_first_sale_hl

In [57]:
X_test = data[data.date_block_num == 34].drop(['date_shop_item_cnt'], axis=1)
Y_test = X_test[['shop_id', 'item_id']]

Y_test = pd.merge(Y_test, X_test_on_sale, on=['shop_id', 'item_id'], how='left')
Y_test = pd.merge(Y_test, X_test_on_sale_hl, on=['shop_id', 'item_id'], how='left')
Y_test = pd.merge(Y_test, X_test_first_sale, on=['shop_id', 'item_id'], how='left')
Y_test = pd.merge(Y_test, X_test_first_sale_hl, on=['shop_id', 'item_id'], how='left')
Y_test

,shop_id,item_id,Y_test_on_sale,date_block_num_x,shop_type_x,item_category_id_x,item_name_tfidf_0_x,item_name_tfidf_1_x,item_name_tfidf_2_x,item_name_tfidf_3_x,...,avg_date_cat_type_cnt_lag_1_y,avg_date_cat_type_cnt_lag_2_y,avg_date_cat_type_cnt_lag_3_y,avg_date_shop_cat_type_cnt_lag_1_y,avg_date_shop_cat_type_cnt_lag_2_y,avg_date_shop_cat_type_cnt_lag_3_y,date_shop_revenue_lag_1_y,date_shop_cat_revenue_lag_1_y,month_y,Y_test_first_sale_hl
0,5,5037,0.481341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,5233,0.811394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,5232,0.452846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,45,18454,0.091844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214196,45,16188,0.017883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214197,45,15757,0.050384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214198,45,19648,0.019880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
def merge_test(row):
    if pd.isnull(row.Y_test_first_sale):
        if pd.isnull(row.Y_test_on_sale_hl):
            return row.Y_test_on_sale
        else:
            return row.Y_test_on_sale_hl
    else:
        if pd.isnull(row.Y_test_first_sale_hl):
            return row.Y_test_first_sale
        else:
            return row.Y_test_first_sale_hl
'''
def merge_test(row):
    if pd.isnull(row.Y_test_first_sale):
        return row.Y_test_on_sale
    else:
        return row.Y_test_first_sale
'''

Y_test['result'] = Y_test.swifter.apply(merge_test, axis=1).fillna(0).astype(np.float16)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [59]:
# load test data set
df_test = pd.read_csv("v30_test_clean.csv")

submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": Y_test.result.values
})

submission.to_csv('v30_submission_stack.csv', index=False)

In [60]:
print("complete!")

complete!


In [ ]:
'''
Y_test_on_sale_lgb = lgb_on_sale.predict(X_test_on_sale).clip(0,20)
Y_test_on_sale_xgb = xgb_on_sale.predict(X_test_on_sale_ft).clip(0,20)
Y_test_on_sale = 0.67*Y_test_on_sale_lgb + 0.33*Y_test_on_sale_xgb

Y_test_on_sale_hl_lgb = lgb_on_sale_hl.predict(X_test_on_sale_hl).clip(0,20)
Y_test_on_sale_hl_xgb = xgb_on_sale_hl.predict(X_test_on_sale_hl_ft).clip(0,20)
Y_test_on_sale_hl = 0.67*Y_test_on_sale_hl_lgb + 0.33*Y_test_on_sale_hl_xgb
'''

In [ ]:
'''
Y_test_first_sale_lgb = lgb_first_sale.predict(X_test_first_sale_ft).clip(0,20)
Y_test_first_sale_xgb = xgb_first_sale.predict(X_test_first_sale_ft_em).clip(0,20)
Y_test_first_sale = 0.67*Y_test_first_sale_lgb + 0.33*Y_test_first_sale_xgb

Y_test_first_sale_hl_lgb = lgb_first_sale_hl.predict(X_test_first_sale_hl_ft).clip(0,20)
Y_test_first_sale_hl_xgb = xgb_first_sale_hl.predict(X_test_first_sale_hl_ft_em).clip(0,20)
Y_test_first_sale_hl = 0.67*Y_test_first_sale_hl_lgb + 0.33*Y_test_first_sale_hl_xgb

'''